In [ ]:
import os
import glob

import pandas as pd

import save_to_bq
from config import *

from datetime import datetime

In [ ]:
voter_file_directory = r'D:\Users\bengen343\Downloads\voter-file'

In [ ]:
# Import Current Voter File
print("Beginning to import current voter file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
allFiles = glob.glob(voter_file_directory + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",", encoding='cp437',index_col=None, header=0, low_memory=False, error_bad_lines=False)
    list_.append(df)
voter_file_df = pd.concat(list_)

voter_file_df.reset_index(drop=True, inplace=True)

# Output the number registered voters
print("Total Registration: {:,}".format(len(voter_file_df)))
print("Finished importing current voter file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
voter_file_df['REGISTRATION_DATE'].max()

In [ ]:
print("Setting column datatypes")
for column in list(voter_file_df):
    if 'DATE' in column:
        voter_file_df[column] = pd.to_datetime(voter_file_df[column], format='%m/%d/%Y', infer_datetime_format=True)
    else:
        voter_file_df[column] = voter_file_df[column].astype('str')
                
    

In [ ]:
print("Building schema")
bq_schema_list = save_to_bq.create_bq_schema(_df=voter_file_df)

print(bq_schema_list)
print(list(voter_file_df))

In [ ]:
bq_table_id = 'cpc-datawarehouse-51210.co_voterfile.2021-06'

In [ ]:
bq_table_id

In [ ]:
print("Uploading to BigQuery")
save_to_bq.save_to_bq(_df=voter_file_df, bq_table_schema=bq_schema_list, table_id=bq_table_id, project_id=bq_project_id)